In [1]:
import pandas as pd
import io
import glob
import re
import nltk
import matplotlib.pyplot as plt
import pymorphy2
import fitz
import string
import numpy as np
import re
import pyLDAvis
import pyLDAvis.gensim
from nltk.stem import *
from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation
from nltk.tokenize import word_tokenize
from tqdm.auto import tqdm, trange
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import pyLDAvis.sklearn
from __future__ import division
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import KMeans
from sklearn.cluster import SpectralClustering
from sklearn.metrics import silhouette_score
from sklearn.manifold import TSNE
from sklearn.decomposition import NMF
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer

__TitleCompany__ - Название компании

__Description__ - Описание компании

__Reiting__ - Рейтинг компании

__Categories__ - Категории компании


__TitleCompany__ - Название компании

__Description__ - Описание компании

__Reiting__ - Рейтинг компании

__Categories__ - Категории компании

__TextPostLemat__ - обработанный, лематизированный текст 

__TextPostToken__ - обработанный, токенизированный и лематизированный текст (лематизирован другим способом нежели TextPostLemat)

__bigram__ - биграммы обработанного и лематизированного текста

__trigram__ - триграммы обработанного и лематизированного текста

__keyword__ - ключевые слова обработанного и лематизированного текста

__clustersAgglomerative__ - кластеры, образованные моделью Agglomerative

__clustersKmeans__ - кластеры, образованные моделью Kmeans

__clustersSpectral__ - кластеры, образованные моделью Spectral

In [ ]:
dfHabr1 = pd.read_csv('Report1.csv')
mystem = Mystem() 
# добавление стопслов
russian_stopwords = stopwords.words("russian")
russian_stopwords.extend(['…', '«', '»', '...', 'быть', 'r', 'n', 'а', 'мы', 'с', 'для', 'ещё', 'его', 'также', 'к', 'тем', 'кто', 'чтобы', 'но', 'они', 'будут', 'так', 'где', 'один', 'он ', 'и', 'на', 'но', 'или', 'либо', 'это', 'мб', 'далее', 'дв', 'свой', 'ваш','всё', 'очень', 'её', 'ещё', 'вообще', 'наш', 'который'])
def tokenize(text):
    t = word_tokenize(text)
    return [token for token in t if token not in russian_stopwords]


dfHabr1['trigram'][i] = list(nltk.ngrams(dfHabr1['TextPostToken'][i], 3))
freq_dist = nltk.FreqDist(dfHabr1['TextPostToken'][i])
    top_words = [word for word, count in freq_dist.most_common(10)]
    dfHabr1['keyword'][i] = ', '.join(top_words)
    
    
def preprocess_text(data, stopwords=russian_stopwords):
    text = re.sub('ё','е', data.lower())
    text = text.strip()
    text = [w for w in text.split() if w not in stopwords] 
    text = [w for w in text if len(w) >=3]
    return ' '.join(text)

def transform_data(data: pd.Series) -> list:
    result = [preprocess_text(data=i) for i in data]
    return result
%%time
dfHabr2 = transform_data(text)
vectorizer = CountVectorizer(analyzer='word', stop_words=russian_stopwords, ngram_range=(1, 3), min_df=1)
count_matrix = vectorizer.fit_transform(dfHabr2)
agg = AgglomerativeClustering(n_clusters=11).fit(count_matrix.toarray())
labelsAgglomerative = agg.labels_
clusters1 = agg.labels_.tolist()
cluster_counts = dfHabr1['clustersAgglomerative'].value_counts()
plt.pie(cluster_counts.values, labels=cluster_counts.index, autopct='%1.1f%%')
plt.show()
a = (dfHabr1['clustersKmeans'] - dfHabr1['clustersKmeans'].mean()) / dfHabr1['clustersKmeans'].std()
data_norm_kmeans = a.values.reshape(-1, 1)
silhouette_avg = silhouette_score(data_norm_kmeans, km.labels_)
print("Silhouette  score:", silhouette_avg)
# создание модели LDA
lda_model = LatentDirichletAllocation(n_components=11, random_state=0)
lda_model.fit(count_matrix)

# вывод топ слов для каждой темы
for i, topic in enumerate(lda_model.components_):
    print(f"Topic {i}: {', '.join([vectorizer.get_feature_names_out()[i] for i in topic.argsort()[:-11:-1]])}")

In [ ]:
# создание матрицы документ-термин
tfidf_vectorizer = TfidfVectorizer(stop_words=russian_stopwords)
tfidf_matrix = tfidf_vectorizer.fit_transform(dfHabr2)

In [ ]:
# создание модели NMF
nmf_model = NMF(n_components=11, random_state=0)
nmf_model.fit(tfidf_matrix)

# вывод топ слов для каждой темы
for i, topic in enumerate(nmf_model.components_):
    print(f"Topic {i}: {', '.join([tfidf_vectorizer.get_feature_names_out()[i] for i in topic.argsort()[:-11:-1]])}")

In [ ]:
pyLDAvis.enable_notebook()
# визуализация модели LDA
panel = pyLDAvis.sklearn.prepare(lda_model, count_matrix, vectorizer, mds='tsne', sort_topics=False)
panel

In [ ]:
topics = []

for i, topic in enumerate(lda_model.components_):
    top_words = [vectorizer.get_feature_names_out()[i] for i in topic.argsort()[:-11:-1]]
    topic_string = f"Topic {i}: {', '.join(top_words)}"
    topics.append(topic_string)
dfCosinTest = pd.DataFrame({'texts': topics})
for i in range(0, 11):
    find_nearest_to = str(dfHabr1.loc[dfHabr1['clustersAgglomerative'] == i, 'keyword'])
    # формирование весов tf-idf
    tfidf = TfidfVectorizer()
    mx_tf = tfidf.fit_transform(topics)
    new_entry = tfidf.transform([find_nearest_to])
    sdf = pd.DataFrame.sparse.from_spmatrix(mx_tf)
    new_entry = tfidf.transform([find_nearest_to])
    pd.DataFrame.sparse.from_spmatrix(new_entry)
    cosine_similarities = cosine_similarity(new_entry, mx_tf).flatten()
    dfCosinTest['cos_similarities'] = cosine_similarities
    dfCosinTest = dfCosinTest.sort_values(by=['cos_similarities'], ascending=[0])
    dfCosinTest = dfCosinTest.reset_index(drop=True)
    dfj['texts'].append(dfCosinTest['texts'][0])
    dfj['cos_similarities'].append(dfCosinTest['cos_similarities'][0])
    dfj['cluster'].append(i)

In [ ]:
sns.catplot(x='TitleCompany', kind='count', data=dfHabr1)
sns.catplot(x='Reiting', kind='count', data=dfHabr1)
sns.catplot(x='Categories', kind='count', data=dfHabr1)
# анализ плотности распределения целевой переменной
sns.kdeplot(dfHabr1['clustersAgglomerative'], shade=True)
# визуализация зависимости кластеров от временных признаков
plt.scatter(dfHabr1['TitleCompany'], dfHabr1['clustersAgglomerative'])
plt.xlabel('Title')
plt.ylabel('Cluster')
plt 